In [3]:
from IPython.display import display, Image, Audio

import cv2  # We're using OpenCV to read video, to install !pip install opencv-python
import base64
import time
from openai import OpenAI
import os
import requests
from dotenv import load_dotenv



In [7]:
load_dotenv()

True

In [8]:
client = OpenAI()

Create Assistant

In [9]:
assistant = client.beta.assistants.create(
  model="gpt-4o",
  instructions="You are an expert illustrator. Your task is to generate images to supplement the story from given input text. Make sure the description of the image is accurate and as specific as possible. Your description should also be family friendly. It is important that each picture generated is consistent with the others. Limit to 2 images",
  tools=[{
      "type": "function",
      "function": {
        "name": "generate_images",
        "description": "Generate image based on the given text.",
        "parameters": {
          "type": "object",
          "properties": {
            "input": {
              "type": "string",
              "description": "Descriptive text of the image to generate"
            },
            "style" : {
              "type" : "string",
              "description" : "The style of image to be generated"
            }
          },
          "required": ["input", "style"]
        }
      }
    },],
  name="Illustrator",
)

Create Thread

In [13]:
thread = client.beta.threads.create()
style = "claymation"
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=f"""
An elderly man was sitting alone on a dark path. He wasn't sure of which direction to go, and he'd forgotten both where he was traveling to...and who he was. He remembered absolutely nothing. He suddenly looked up to see an elderly woman before him.

She grinned toothlessly and with a cackle, spoke: "Now your third wish. What will it be?"

"Third wish?" The man was baffled. "How can it be a third wish if I haven’t had a first and second wish?"

"You’ve had two wishes already," the hag said, "but your second wish was for you to forget everything you know." She cackled at the poor man. "So it is that you have one wish left."

"All right," he said hesitantly, "I don't believe this, but there's no harm in trying. I wish to know who I truly am."

"Funny," said the old woman as she granted his wish and disappeared forever. "That was your first wish..."

Illustrate this story in {style} style
"""
)

Dall-E

In [11]:
def generate_images(input, style):
  try:
    dalle = client.images.generate(
      model="dall-e-3",
      prompt=f"Generate image in a {style} style. DO NOT add any detail, just use it AS-IS. {input}",
      size="1024x1024",
      quality="standard",
      n=1
    )
    return dalle.data
  except:
    return None

In [14]:
from typing_extensions import override
from openai import AssistantEventHandler
import json
f = open("output/test.txt", "a")
class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
      # Retrieve events that are denoted with 'requires_action'
      # since these will have our tool_calls
      if event.event == 'thread.run.requires_action':
        run_id = event.data.id  # Retrieve the run ID from the event data
        for tool in event.data.required_action.submit_tool_outputs.tool_calls:
          print(f"Image description : {tool.function.arguments}")
        self.handle_requires_action(event.data, run_id)
 
    def handle_requires_action(self, data, run_id):
      tool_outputs = []
        
      for tool in data.required_action.submit_tool_outputs.tool_calls:
        if tool.function.name == "generate_images":
          arguments = json.loads(tool.function.arguments)
          prompt = arguments["input"]
          style = arguments["style"]
          images = generate_images(prompt, style)
          if (images != None):
            for image in images:
              tool_outputs.append({"tool_call_id": tool.id, "output": image.url})
          else :
            tool_outputs.append({"tool_outputs" : tool.id, "output" : "error generating image"})
        
      # Submit all tool_outputs at the same time
      self.submit_tool_outputs(tool_outputs, run_id)
 
    def submit_tool_outputs(self, tool_outputs, run_id):
      # Use the submit_tool_outputs_stream helper
      with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=self.current_run.thread_id,
        run_id=self.current_run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler(),
      ) as stream:
        for text in stream.text_deltas:
          f.write(text)
          print(text, end="", flush=True)
        print()
 
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler()
) as stream:
  stream.until_done()

f.close()

Image description : {"input": "An elderly man sitting alone on a dark path, looking confused and disoriented. The setting is eerie with twisted trees and a misty atmosphere. The man appears aged with wrinkles and white hair, wearing simple, worn-out clothes.", "style": "claymation"}
Image description : {"input": "An elderly woman with a toothless grin and a cackling expression, standing before the elderly man. She is dressed in tattered clothing with a crooked posture, appearing almost witch-like. The path remains dark and eerie with twisted trees and mist.", "style": "claymation"}
Here are the illustrations for the story in claymation style:

1. **Elderly Man on a Dark Path**
   ![Elderly Man on a Dark Path](https://oaidalleapiprodscus.blob.core.windows.net/private/org-TcLC4MzeW49IpVv7cXWgseJh/user-U3n9QgtiKflsUkg7hjpKR7ky/img-NSskcrYq6SHXDA9jz2aFRjJM.png?st=2024-07-02T02%3A38%3A40Z&se=2024-07-02T04%3A38%3A40Z&sp=r&sv=2023-11-03&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698

In [19]:
style = "claymation"
input = f"""An elderly man was sitting alone on a dark path. He wasn't sure of which direction to go, and he'd forgotten both where he was traveling to...and who he was. He remembered absolutely nothing. He suddenly looked up to see an elderly woman before him.

She grinned toothlessly and with a cackle, spoke: "Now your third wish. What will it be?"

"Third wish?" The man was baffled. "How can it be a third wish if I haven’t had a first and second wish?"

"You’ve had two wishes already," the hag said, "but your second wish was for you to forget everything you know." She cackled at the poor man. "So it is that you have one wish left."

"All right," he said hesitantly, "I don't believe this, but there's no harm in trying. I wish to know who I truly am."

"Funny," said the old woman as she granted his wish and disappeared forever. "That was your first wish..."

Illustrate this story in {style} style"""

In [20]:
genie.create_thread(input)

AttributeError: 'Genie' object has no attribute 'create_thread'